In [ ]:
from shapely.geometry import MultiPolygon, Polygon
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import os
import csv

In [ ]:
file = "..\\rawData\\countries.geoJson"

with open(file, 'r') as f:
    data = json.load(f)

In [ ]:
# find index of russia
for i, feature in enumerate(data['features']):
    if feature['properties']['name'] == 'Australia':
        idx = i
        break

print(f"Index : {idx}")

In [ ]:
tolerance = 0
idx = 90

feature = data['features'][idx]
iso3 = feature['properties']['ISO3166-1-Alpha-3']
geometry = shape(feature['geometry'])
simplified_geom = geometry.simplify(tolerance, preserve_topology=True)

# Ensure geometry is iterable
if isinstance(simplified_geom, Polygon):
    polygons = [simplified_geom]
elif isinstance(simplified_geom, MultiPolygon):
    polygons = list(simplified_geom.geoms)
else:
    raise TypeError(f"Unsupported geometry type: {simplified_geom.geom_type}")

outline_coords = []

for polygon in polygons:
    outline = polygon.boundary
    if outline.geom_type == 'MultiLineString':
        outline_coords.extend([list(line.coords) for line in outline.geoms])
    else:
        outline_coords.append(list(outline.coords))


In [ ]:
# plot the outline
import matplotlib.pyplot as plt
for coords in outline_coords:
    x, y = zip(*coords)
    plt.plot(x, y)

plt.title(f"Outline of {iso3} with tolerance {tolerance}")

In [ ]:
import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon

tolerance = 0
idx = 90

# Convert the feature to a GeoDataFrame
feature = data['features'][idx]
geom = shape(feature['geometry'])
iso3 = feature['properties']['ISO3166-1-Alpha-3']

gdf = gpd.GeoDataFrame({'geometry': [geom]}, crs="EPSG:4326")  # Assuming original is WGS84

# Reproject to EPSG:3857
gdf = gdf.to_crs(epsg=3857)

# Simplify geometry (in meters, since EPSG:3857 units)
simplified_geom = gdf.geometry.iloc[0].simplify(tolerance, preserve_topology=True)

# Now handle MultiPolygon or Polygon
if isinstance(simplified_geom, Polygon):
    polygons = [simplified_geom]
elif isinstance(simplified_geom, MultiPolygon):
    polygons = list(simplified_geom.geoms)
else:
    raise TypeError(f"Unsupported geometry type: {simplified_geom.geom_type}")

outline_coords = []

for polygon in polygons:
    # Only take exterior ring coordinates, ignoring holes
    outline_coords.append(list(polygon.exterior.coords))


In [ ]:
# plot the outline
import matplotlib.pyplot as plt
for coords in outline_coords:
    x, y = zip(*coords)
    plt.plot(x, y)

plt.title(f"Outline of {iso3} with tolerance {tolerance}")

In [ ]:
# load jsoin file
import json
file1 = "../processedData/json/epsg4326/tol0/ITA.json"
file2 = "../processedData/json/epsg3857/tol0/ITA.json"

with open(file1, 'r') as f:
    data1 = json.load(f)

with open(file2, 'r') as f:
    data2 = json.load(f)

In [ ]:
# plot the outline
import matplotlib.pyplot as plt
for coords in data1['coordinates']:
    x, y = zip(*coords)
    plt.plot(x, y, label='EPSG:4326')

In [ ]:
for coords in data2['coordinates']:
    x, y = zip(*coords)
    plt.plot(x, y, label='EPSG:3857')

In [ ]:
## CITY

In [ ]:
from pyproj import Transformer

# Define the transformer: WGS84 (EPSG:4326) to Web Mercator (EPSG:3857)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Coordinates of Rome: (lon, lat)
lon, lat = 12.4822, 41.8967

# Transform to EPSG:3857
x, y = transformer.transform(lon, lat)

print(f"Rome in EPSG:3857: x={x:.2f}, y={y:.2f}")


In [ ]:
# plot italy and rome
for coords in data2['coordinates']:
    x, y = zip(*coords)
    plt.plot(x, y, label='EPSG:3857')

plt.scatter(x, y, color='red', label='Rome (EPSG:3857)', s=100, edgecolor='black')

In [ ]:
## Json and cities
file = "..\\rawData\\countries.geoJson"

with open(file, 'r') as f:
    data = json.load(f)

In [ ]:
from shapely.ops import transform

idx = 90

feature = data['features'][idx]
iso3 = feature['properties']['ISO3166-1-Alpha-3']
geometry = shape(feature['geometry'])
    
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
geometry = transform(transformer.transform, geometry)


outline_coords = []
for polygon in polygons:
    outline = polygon.boundary
    if outline.geom_type == 'MultiLineString':
        outline_coords.extend([list(line.coords) for line in outline.geoms])
    else:
        outline_coords.append(list(outline.coords))

# plot
# Define the transformer: WGS84 (EPSG:4326) to Web Mercator (EPSG:3857)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Coordinates of Rome: (lon, lat)
lon, lat = 12.4822, 41.8967

# Transform to EPSG:3857
rome_x, rome_y = transformer.transform(lon, lat)

print(f"Rome in EPSG:3857: x={rome_x:.2f}, y={rome_y:.2f}")


import matplotlib.pyplot as plt
for coords in outline_coords:
    x, y = zip(*coords)
    plt.plot(x, y)
    
plt.scatter(rome_x, rome_y, color='red', label='Rome (EPSG:3857)', s=100, edgecolor='black')

In [ ]:
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely.ops import transform
from pyproj import Transformer
import json
import os

# --- SETTINGS ---
file = "../rawData/countries.geoJson"
idx_list = [47, 90, 107]  # Example country indices
tolerance = 0
normalize = False
scale_factor = 130478800.65  
crs = "EPSG:3857"
output_dir = "../processedData/test"

# --- LOAD DATA ---
with open(file, 'r') as f:
    data = json.load(f)

# --- TRANSFORMER (WGS84 -> Web Mercator) ---
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# --- LOOP OVER SELECTED COUNTRIES ---
for idx in idx_list:
    feature = data['features'][idx]
    iso3 = feature['properties']['ISO3166-1-Alpha-3']
    geometry = shape(feature['geometry'])

    # --- Reproject to EPSG:3857 ---
    geometry = transform(transformer.transform, geometry)

    # --- Simplify geometry ---
    simplified_geom = geometry.simplify(tolerance, preserve_topology=True)

    # --- Make polygons iterable ---
    if isinstance(simplified_geom, Polygon):
        polygons = [simplified_geom]
    elif isinstance(simplified_geom, MultiPolygon):
        polygons = list(simplified_geom.geoms)
    else:
        raise TypeError(f"Unsupported geometry type: {simplified_geom.geom_type}")

    # --- Compute GLOBAL NORMALIZATION bounds ---
    # (we do it per-country here, but will replace this with global later)
    if normalize:
        minx, miny, maxx, maxy = simplified_geom.bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2
        scale = 1 / scale_factor

    # --- Extract and normalize outlines ---
    outline_coords = []

    for polygon in polygons:
        outline = polygon.boundary
        lines = outline.geoms if outline.geom_type == 'MultiLineString' else [outline]
        for line in lines:
            coords = list(line.coords)
            if normalize:
                coords = [((x - center_x) * scale, (y - center_y) * scale) for x, y in coords]
            outline_coords.append(coords)

        print(outline_coords)
    

    # --- Save output ---
    iso_dir = os.path.join(output_dir, f'{iso3}')
    os.makedirs(iso_dir, exist_ok=True)
    out_path = os.path.join(iso_dir, f"{iso3}_tol{str(tolerance).replace('.', '_')}.json")

    with open(out_path, 'w') as f:
        json.dump({"coordinates": outline_coords}, f, indent=2)

    print(f"[✓] Saved {iso3} to {out_path}")


In [ ]:
geometry = transform(transformer.transform, geometry)
print(f"[DEBUG] {iso3} bounds in EPSG:3857:", geometry.bounds)


In [ ]:
# plot all json 
path = '../processedData/test'

import matplotlib.pyplot as plt 
import json
import os

# plot all files in path 
files = [f for f in os.listdir(path) if f.endswith('.json')]
for file in files:
    with open(os.path.join(path, file), 'r') as f:
        data2 = json.load(f)
    for coords in data2['coordinates']:
        x, y = zip(*coords)
        plt.plot(x, y, label=file)
plt.xlabel('Longitude (EPSG:4326)')
plt.ylabel('Latitude (EPSG:4326)')
plt.title('Countries in EPSG:4326')
plt.legend()


